Un lazo PID (Proporcional-Integral-Derivativo) es un tipo de controlador utilizado en sistemas de control industrial para regular variables de proceso, como temperatura, presión, velocidad, entre otros. El objetivo de un controlador PID es mantener una variable de proceso en un valor deseado, conocido como setpoint, minimizando el error entre el valor real de la variable y el setpoint.

In [9]:
import time

class bomba:
    def __init__(self, MaxRPM) -> None:
        self.MaxRPM = MaxRPM #velocidad maxima

        self.RPM = 0 #RPM actuales
        self.Valvula = 0

        self.anteriores=[] #necesario para que el PID funcione


    def update(self, tiempo):
        # Limitamos entre 0 y velocidad máxima 
        self.RPM += (-10 + (self.Valvula*1.8)) * tiempo #Si la valvula está cerrada, pierde velocidad

        if self.RPM < 0:
            self.RPM = 0
        elif self.RPM > self.MaxRPM:
            self.RPM = self.MaxRPM
        
# ------------------ AQUI VA EL PID (dentro de la clase, ojo la indentación) ------------------
    def PID(self,input, Man_Auto = False, SetpointMan = 0.0, SetpointAuto = 0.0):
            """
                Calcula la salida de un controlador PID (Proporcional, Integral, Derivativo).

                Args:
                    Man_Auto (bool): Modo manual (True) o automático (False).
                    SetpointMan (bool): Ignorado si Man_Auto es True. Modo manual de setpoint (True) o automático (False).
                    SetpointAuto (float): El valor del setpoint en modo automático.

                Returns:
                    None

                El método calcula la salida del controlador PID utilizando el valor actual (input) y el setpoint (SetpointAuto).
                Se almacena el histórico de velocidades en self.anteriores y se limita a 100 elementos.
                Se calculan los componentes P, I y D del controlador y se suman para obtener la salida.
                La salida se limita al rango de 0 a 100.

                """
            if Man_Auto == False:     
                # Si el PID está en modo automático...

                # Almaceno el vector velocidad en una lista de 100 elementos.
                self.anteriores.append(input)
                if len(self.anteriores) > 100:
                    self.anteriores = self.anteriores[-100:]

                SP = SetpointAuto        
                E = SP - input
                self.error = E 
                
                #error es la diferencia entre lo que tengo, y mi setpoint actual. Usamos la lista para ello. 
                E_accu = [(SP - elem) for elem in self.anteriores[-20:]]
                self.error_accu = E_accu
                
                kP = 10.0
                kI = 0.0001
                kD = 0.01

                #La acción proporcional es el error multiplicado por una constante
                aP = self.error * kP
                
                #La acción integral es el área de los valores, dividido por la constante
                aI = (kP * (sum(self.error_accu) / (len(self.error_accu)*0.002) * kI))

                #La acción derivativa es la proyección a futuro (pendiente) del error, multiplicado por una constante
                if len(self.anteriores)>2:
                    aD = (self.error_accu[-1]-self.error_accu[-2])*kD*kP
                else:
                    aD = 0.0
                
                #sumamos las componentes de las acciones Proporcional, Integral y Derivativa
                Salida = self.Valvula + aP + aI + aD

                #Limitamos la válvula de salida
                if Salida < 0:
                    self.Valvula = 0
                elif Salida > 100:
                    self.Valvula = 100         
                else:
                    self.Valvula = Salida

            else:
                # Si estamos en modo "Manual", la válvula se pone en la posición que definimos en el setpoint.
                self.Valvula = SetpointMan


# ------------- COMIENZA PROGRAMA ----------------
M1 = bomba(MaxRPM=4000)
consigna = 1500 # la velocidad a la que quiero que llegue
tiempo = 0.05 # tiempo de update en segundos

try:
    while True:
        # Quitarle el # a lo que quieras probar:
        # --------------------------------------
        # Nota: En modo manual, el lazo NO regula, entonces se "pasa" de la velocidad pedida. Es como si
        # nosotros controlaramos la apertura de la válvula con una perilla, a mano.

        # 1. PID manual, válvula al 30% fija
        M1.PID(M1.RPM, Man_Auto=True,SetpointMan=30)
        # 2. PID manual, válvula al 0% fija
        #M1.PID(M1.RPM, Man_Auto=True,SetpointMan=0)
        # 3. PID manual, válvula al 100% fija
        #M1.PID(M1.RPM, Man_Auto=True,SetpointMan=100)

        #En modo automático, la válvula se controla sola, sin intervención.
        # 4. PID automático
        #M1.PID(M1.RPM, Man_Auto=False,SetpointAuto=consigna)

        # ---------------- Uodate y print ---------------
        M1.update(tiempo)
        print(f"Consigna: {consigna} - Velocidad real: {M1.RPM:.2f} - Posicion comando: {M1.Valvula:.2f}           ",end='\r')
        time.sleep(tiempo)

except KeyboardInterrupt:
    print("Simulación terminada.")

Simulación terminada.cidad real: 4000.00 - Posicion comando: 30.00           
